In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re

# --- Configuration ---
CSV_FILE_PATH = 'SampleSuperstore.csv'
DATABASE_FILE_NAME = 'superstore.db'
TABLE_NAME = 'sales'

def clean_col_names(df):
    """
    Cleans column names to be valid SQL identifiers.
    - Converts to lowercase
    - Replaces spaces and hyphens with underscores
    - Removes any other non-alphanumeric characters (except underscores)
    """
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = col.lower()
        new_col = new_col.replace(' ', '_').replace('-', '_')
        new_col = re.sub(r'[^a-zA-Z0-9_]', '', new_col)
        new_cols.append(new_col)
    df.columns = new_cols
    return df

def create_database():
    """
    Reads data from a CSV file and loads it into a new SQLite database table.
    """
    print(f"--- Starting Database Creation from '{CSV_FILE_PATH}' ---")
    
    # --- 1. Read and Clean the CSV Data ---
    try:
        df = pd.read_csv(CSV_FILE_PATH, encoding='windows-1252')
        # This dataset is known to have a non-standard encoding.
        # 'windows-1252' or 'latin1' often works if 'utf-8' fails.
        
        print("Successfully read CSV file.")
        
        # Clean column names for SQL compatibility
        df = clean_col_names(df)
        print("Cleaned column names for database compatibility:")
        print(df.columns.tolist())
        
    except FileNotFoundError:
        print(f"Error: The file '{CSV_FILE_PATH}' was not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the CSV: {e}")
        return

    # --- 2. Create SQLAlchemy Engine and Write to DB ---
    try:
        # The connection string for a SQLite database is 'sqlite:///filename.db'
        engine = create_engine(f'sqlite:///{DATABASE_FILE_NAME}', echo=False)
        
        print(f"\nWriting data to table '{TABLE_NAME}' in database '{DATABASE_FILE_NAME}'...")
        
        # Use df.to_sql to create the table and insert the data.
        # if_exists='replace': If the table already exists, drop it and create a new one.
        # index=False: Do not write the DataFrame's index as a column.
        df.to_sql(TABLE_NAME, con=engine, if_exists='replace', index=False)
        
        print("\n--- Success! ---")
        print(f"Database '{DATABASE_FILE_NAME}' and table '{TABLE_NAME}' created successfully.")
        
    except Exception as e:
        print(f"An error occurred during database creation: {e}")

if __name__ == '__main__':
    create_database()


--- Starting Database Creation from 'SampleSuperstore.csv' ---
Successfully read CSV file.
Cleaned column names for database compatibility:
['ship_mode', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'category', 'sub_category', 'sales', 'quantity', 'discount', 'profit']

Writing data to table 'sales' in database 'superstore.db'...

--- Success! ---
Database 'superstore.db' and table 'sales' created successfully.
